In [4]:
%pylab inline
import numpy as np

num_egs = 5
num_features = 2
num_classifiers = 3

X = np.zeros((num_egs,num_features))
labels = np.zeros(num_egs)
X[0][0]=10; labels[0] = 1;
X[1][0]=30; labels[1] = 1;
X[2][0]=40; labels[2] = -1;
X[3][0]=60; labels[3] = -1;
X[4][0]=90; labels[4] = 1;

X[0][1]=40; 
X[1][1]=50; 
X[2][1]=40; 
X[3][1]=50; 
X[4][1]=40; 

# X[0][0]=10; labels[0] = 1;
# X[1][0]=20; labels[1] = -1;
# X[2][0]=30; labels[2] = 1;


Populating the interactive namespace from numpy and matplotlib


/home/praateek/miniconda3/envs/twitter/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [17]:
# C++ like data structure
class Decision_Direction_Error():
    def __init__(self, error, direction):
        self.error = error
        self.direction = direction

class DecisionStump():
    def __init__(self, criteria='gini'):
        self.criteria = criteria

    
    def get_error_curr_feature_val(labels, weights, feature_vals, split_value):
        for i in range(self.weights):
            # x[i] < c should be -1
            
            if ( ((feature_vals[i] <= split_value) and (labels[i]!=-1)) or 
                ((feature_vals[i] > split_val) and (labels[i]!=1)))
            {
                curr_error += weights[i];
            }
            
            sol = Decision_Direction_Error(curr_error, 1)
            
            if curr_error < 0.5:
                sol.direction = 1
                sol.error = curr_error
            else:
                
        
    
    def get_feature_threshold_curr_feature(labels, weights, feature_vals, unique_feature_vals):
        for i in range(len(unique_feature_vals)):
            self.get_error_curr_feature_val(labels, weights, feature_vals, unique_feature_vals[i])
        
    def get_best_feature_stump(self, labels, X_T, weights unique_feature_vals):
        
        self.num_features = len(X_T)
        self.num_egs = len(X_T[0])
        
        best_error = float('inf')
        
        for i in range(self.num_features):
            best_stump = get_feature_threshold_curr_feature(labels, weights, feature_vals[i], 
                                                            unique_feature_vals[i])
        

            
    def fit(X, y):
        num_egs = len(X)
        num_features = len(X[0])
        # Let's sort each feature by value initially
        # This way we don't have to sort every time
        for feature in range(num_features):
            indices = np.argsort( X[:,feature] );
            X_temp = X[indices]
            

class AdaBoost():
    def __init__(self, num_classifiers='3'):
        self.num_classifiers = num_classifiers
    
    def get_unique_feature_vals(self,X_T):
        '''
            Args:
                x : dataset
                
            Returns:
                feature_midpoints : For each feature returns a sorted list of mid points 
                                    of every two consecutive unique features
            
            Complexity:
               (Old) :  M + N * M + M * NlogN + M * N
               (New) : M * N log N + M * N
               
        '''

        
        X_copy = []     
        feature_midpoints = []

        # M * n log * n
        for i in range(self.num_features):
            X_copy.append(sorted(X_T[i]))
        
        # Get mid points of sorted unique values M * N
        for i in range(self.num_features):
            current = []
            j = 0
            while (j < self.num_egs - 1):
                k = j + 1
                while(X_copy[i][j] == X_copy[i][k] and k < self.num_egs - 1):
                    k += 1
                midpoint = (X_copy[i][j] + X_copy[i][k])/2
                current.append(midpoint)
                j = k
            feature_midpoints.append(current)
                            
        return feature_midpoints
    
    def get_feature_threshold_curr_feature():
        
    
    
    def fit_weak_classifers(self, labels, X_T, unique_feature_vals):
        weights = np.ones(self.num_egs)/len(self.num_features)
        classifiers = []
        for t in range(self.num_classifiers):
            clf = self.get_best_feature_stump(labels, X_T, weights, unique_feature_vals);
    
    def fit(self, X, y):
        self.num_egs = len(X)
        self.num_features = len(X[0])
        X_T = X.T
        classifiers = []
        unique_feature_vals = self.get_unique_feature_vals(X_T)
            
#         for t in range(self.num_classifiers):
            

In [18]:
clf = AdaBoost()
clf.fit(X,labels)

In [41]:
A = [1,1,1,1,1,2,2,2,3,3,3,4]
j = 0
while (j < len(A) - 1):
    k = j + 1
    while(A[j] == A[k]):
        k += 1
    print((A[j] + A[k])/2)
#     print(j,k)
    j = k


1.5
2.5
3.5
